# 1. 去除主题，情感词都为空的行

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_excel('./data/origin.xlsx')
data = data.fillna("$$$")
duty_data = data[data.sentiment_word == "$$$"]
clean_data = data.drop(data[data.sentiment_word == "$$$"].index)

# 情感词为空的数据
duty_data.to_csv("./data/duty_data.csv", sep='\t', index=None, encoding="utf8")
# 干净的数据
clean_data.to_csv("./data/clean_data.csv", sep='\t', index=None, encoding="utf8")

# 2. 划分训练集和测试集

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
clean_data = pd.read_csv("./data/clean_data.csv", sep='\t')
data = clean_data.sample(frac = 1)
percent = int(data.shape[0] * 0.9)
train_data = data.iloc[:percent]
test_data = data.iloc[percent:]

In [ ]:
train_data.to_csv("./data/train.csv", sep="\t", index=None)
test_data.to_csv('./data/test.csv', sep="\t", index=None)

# 3. 检测content分词后，是否能保留情感词

In [ ]:
import thulac
import numpy as np
import pandas as pd
train_data = pd.read_csv('./data/clean_data.csv', sep='\t')
seg = thulac.thulac(user_dict='./data/test.txt')
# row = train_data.iloc[1]
# content = row["content"]
# sentiment_words = row["sentiment_word"]



# content_data = train_data.loc[:,'content']

# #使用ltp依次进行分词、词性标注、句法分析
# # -*- coding: utf-8 -*-
# from pyltp import Segmentor
# from pyltp import Postagger
# from pyltp import NamedEntityRecognizer
# from pyltp import Parser

# import os
# LTP_DATA_DIR = '/home/zwl/pyltp-master/ltp_data'  # ltp模型目录的路径
# cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`

# #分词
# segmentor = Segmentor()  # 初始化实例
# segmentor.load_with_lexicon(cws_model_path, './data/test.txt')  # 加载模型




In [ ]:
seg_res = []
word_res = []
cnt_res = []
def lexcial_coverage(content, words):
    seg_words = seg.cut(content)
#     print(list(seg_words))
    no_pos_words = [item[0] for item in seg_words ]
    seg_res.append(no_pos_words)
    words_list = words.split(';')
    word_res.append(words_list)
#     print(no_pos_words)
#     print(words_list)
#     print('******\n')
    cnt = 0
    for word in words_list:
        try:
            tmp = no_pos_words.index(word)
            cnt = cnt + 1
        except:
            pass
    cnt_res.append(cnt)
    return cnt / len(words_list)
res = 0
i = 0
for row in train_data.values:
    print(i)
    i = i + 1
    content = row[1]
    words = row[3]
    percent = lexcial_coverage(content, words)
    res = res + percent


In [ ]:
seg_series = pd.Series(seg_res)
word_series = pd.Series(word_res)
cnt_series = pd.Series(cnt_res)

In [ ]:
def delete(s):
    res = ''
    i = 0
    for item in s:
        if i == 0:
            res = res + item
        else:
            res = res + ',' + item
    return res

In [ ]:
haha = pd.DataFrame({"seg":seg_series, "word":word_series, "cnt":cnt_series})
def delete(s):
    res = ''
    i = 0
    for item in s:

        if i == 0:
            res = res + item
        else:
            res = res + '\t' + item
        i = i + 1
    return res
haha["seg"] = haha["seg"].apply(delete)
haha["word"] = haha["word"].apply(delete)

# 4. 重新对content分词, 生成序列标注的数据格式

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv('./data/validation.csv', delimiter=',')
content = pd.DataFrame(data["content"])
content.to_csv('./data/validation_content.csv', header=None, index=None)
import thulac
model = thulac.thulac(seg_only=True, user_dict='./data/dict.txt')
model.cut_f(input_file='./data/validation_content.csv', output_file='./data/validation_seg_content.csv')

In [ ]:
seg_content = pd.read_csv('./data/validation_seg_content.csv', delimiter='\t', header=None)

sentiment_set = pd.read_csv('./data/sentiment_set.csv', header=None)
theme_set = pd.read_csv('./data/theme_set.csv', header=None)


sentiment = set()
for item in sentiment_set.values:
    sentiment.add(item[0])
theme = set()
for item in theme_set.values:
    theme.add(item[0])

def list_to_str(l):
    res = ""
    for i in range(len(l)):
        if i == 0:
            res = res + l[i]
        else:
            res = res + " " + l[i]
    return res
haha = []
with open('./data/validation_seg_content.csv', 'r') as f:
    for (num,value) in enumerate(f):
        words = value.split(' ')
        res = []
        pos = 0
        for word in words:
            if word in sentiment:
                res.append('S')
            elif word in theme:
                res.append('T')
            else:
                res.append('O')
        target = [list_to_str(res)]
        haha.append(target)



haha = pd.DataFrame(haha)

haha

haha.to_csv('./data/validation_target.csv', header=None, index=None)

In [ ]:
res = ['S', 'T', 'O']

In [ ]:
with open('./data/target.csv', 'w') as target:
    target.write(str(res))
